# Word2Vec

## Introducción

Word2Vec es una técnica que se utiliza para obtener representaciones vectoriales de palabras. Estas representaciones son útiles para realizar tareas de procesamiento de lenguaje natural, como la clasificación de texto, la traducción automática, la generación de texto, etc. Su objetivo es capturar las relaciones semánticas y sintácticas entre palabras basándose en su contexto dentro de un corpus de texto. Fue introducido por Mikolov et al. en 2013.

Esta técnica se basa en la hipótesis distribucional, que establece que las palabras que aparecen en contextos similares tienen significados similares. Por lo tanto, Word2Vec se entrena para predecir las palabras vecinas de una palabra dada, utilizando un corpus de texto como datos de entrenamiento.

Tiene dos variantes principales: Skip-gram y Continuous Bag of Words (CBOW). La diferencia entre ambas radica en la forma en que se plantea el problema de predicción. En el caso de Skip-gram, se predice el contexto a partir de una palabra dada, mientras que en CBOW se predice la palabra a partir de su contexto.

Por ejemplo en el texto "El gato come pescado", si utilizamos una ventana de tamaño 2, el contexto de la palabra "come" sería "El gato pescado". En el caso de Skip-gram, se trataría de predecir "El", "gato", "pescado" a partir de "come":

| Entrada | Salida |
|---------|--------|
| come    | El     |
| come    | gato   |
| come    | pescado|

Mientras que en el caso de CBOW, se trataría de predecir "come" a partir de "El", "gato", "pescado":

| Entrada | Salida |
|---------|--------|
| El      | come   |
| gato    | come   |
| pescado | come   |

En este notebook, vamos a utilizar la implementación de Word2Vec de la librería Gensim para entrenar un modelo de Skip-gram y explorar las representaciones vectoriales de palabras obtenidas.

La **representación inicial de palabras** en Word2Vec es una representación **discreta y esparsa** conocida como **one-hot encoding**. Es el punto de partida antes de que las palabras sean transformadas en vectores densos y continuos mediante el aprendizaje en Word2Vec. Vamos a desglosarlo con ejemplos para clarificar cómo funciona.

---

### 1. **One-Hot Encoding: Representación Básica**
Dado un vocabulario de $ V $ palabras, asignamos un índice único a cada palabra. Luego, representamos cada palabra como un vector de dimensión $ V $, donde todas las posiciones son 0 excepto una, que es 1.

#### Ejemplo Simple
Supongamos un vocabulario con las siguientes palabras:
$$
\text{Vocabulario} = \{\text{gato, perro, ratón, queso}\}
$$

Asignamos índices:
- $ \text{gato} \to 0 $
- $ \text{perro} \to 1 $
- $ \text{ratón} \to 2 $
- $ \text{queso} \to 3 $

La representación **one-hot** para cada palabra será:
$$
\text{gato} = [1, 0, 0, 0], \quad \text{perro} = [0, 1, 0, 0], \quad \text{ratón} = [0, 0, 1, 0], \quad \text{queso} = [0, 0, 0, 1]
$$

---

### 2. **Propiedades del One-Hot Encoding**
1. **Alta Dimensionalidad**:
   Si el vocabulario tiene $ V = 10,000 $ palabras, cada palabra estará representada por un vector de tamaño $ 10,000 $, con un único valor de 1.

2. **Discretas y Esparzas**:
   Solo un elemento del vector es 1, mientras que el resto son ceros. Esto significa que la representación es **esparsa** y ocupa mucho espacio de memoria.

3. **Sin Captura de Relación Semántica**:
   La representación one-hot no tiene en cuenta ninguna relación entre palabras. Por ejemplo:
   - $ \text{gato} = [1, 0, 0, 0] $
   - $ \text{perro} = [0, 1, 0, 0] $

   Aunque $ \text{gato} $ y $ \text{perro} $ están semánticamente relacionados (ambos son animales), sus representaciones one-hot son **ortogonales** y no comparten ninguna similitud matemática.

---

### 3. **Limitaciones del One-Hot Encoding**
#### A. **Dimensión del Vector**
A medida que el vocabulario crece, la dimensión $ V $ se vuelve enorme, lo que hace que esta representación sea ineficiente.

#### B. **Falta de Información Semántica**
La distancia entre vectores no refleja similitudes entre palabras. Por ejemplo:
$$
\text{Distancia Euclidiana}(\text{gato}, \text{perro}) = \text{Distancia Euclidiana}(\text{gato}, \text{queso}) = \sqrt{2}
$$
Esto significa que $ \text{gato} $ está tan "cerca" de $ \text{perro} $ como de $ \text{queso} $, lo cual no es intuitivo.

#### C. **Escalabilidad en Modelos**
Los modelos de aprendizaje automático necesitan procesar estos vectores de alta dimensionalidad, lo cual aumenta la complejidad computacional y limita el rendimiento.

---

### 4. **Transición a Representaciones Densas**
La motivación de Word2Vec es superar estas limitaciones al transformar las palabras en **vectores densos** y de menor dimensión ($ d $) que capturen relaciones semánticas. Por ejemplo:

- Representación inicial (one-hot): $ \text{gato} = [1, 0, 0, 0] $
- Representación Word2Vec: $ \text{gato} = [0.2, -0.4, 0.8, 0.1, -0.3] $

---

### 5. **Ejemplo de Representación Inicial en un Contexto Real**
Imaginemos una oración: 
$$
\text{"El gato persigue al ratón"}
$$

1. Vocabulario:
   $$
   \{\text{el, gato, persigue, al, ratón}\}
   $$

2. Índices de las palabras:
   - $ \text{el} \to 0 $
   - $ \text{gato} \to 1 $
   - $ \text{persigue} \to 2 $
   - $ \text{al} \to 3 $
   - $ \text{ratón} \to 4 $

3. Representaciones one-hot:
   - $ \text{el} = [1, 0, 0, 0, 0] $
   - $ \text{gato} = [0, 1, 0, 0, 0] $
   - $ \text{persigue} = [0, 0, 1, 0, 0] $
   - $ \text{al} = [0, 0, 0, 1, 0] $
   - $ \text{ratón} = [0, 0, 0, 0, 1] $

Cuando esta oración pasa por un modelo como Word2Vec, estas representaciones iniciales se transforman en vectores densos que reflejan las relaciones semánticas entre palabras. Por ejemplo:

$$
\text{gato} \approx [0.3, -0.2, 0.8], \quad \text{ratón} \approx [0.2, -0.1, 0.7]
$$

Estas nuevas representaciones permiten cálculos más avanzados, como medir la similitud semántica entre palabras.

In [ ]:
### Librerias necesarias Word2Vec

import gensim
from gensim.models import Word2Vec

import numpy as np
import pandas as pd
import re
import os

## 2. Cargar el texto

Para este ejercicio inicial usaremos un texto sencillo, el cual se cargará en una variable.

In [ ]:
texto_1 = """Hace varios años, en el pelotón de fusilamiento, el coronel Aureliano Buendía había de recordar aquella tarde remota en que su padre lo llevó a conocer el hielo. """
texto_2 = """Hace tanto tiempo que no me acuerdo de nada, pero recuerdo que mi padre me llevó a conocer el hielo. """
texto_3 = """Hace tiempo que ocurrió la era de hielo, ahorita que solo soy un  perezoso recuerdo aquellos días tan bellos con mis amigos, un mamut y un dientes de sable. """

corpus = [texto_1, texto_2, texto_3]

## 3. Preprocesamiento

El preprocesamiento es una etapa importante en el procesamiento de lenguaje natural. En esta etapa se eliminan las palabras que no aportan información, como los signos de puntuación, las palabras vacías, etc.

In [ ]:
############# Preprocesamiento de texto ####################
def preprocesamiento(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = texto.split()

    return texto

corpus_procesado = [preprocesamiento(texto) for texto in corpus]

print(corpus_procesado)

# **4. Arquitectura de Word2Vec**

## **CBOW (Continuous Bag of Words)**

CBOW busca predecir una palabra objetivo $ w_t $ a partir de sus palabras de contexto. Este proceso transforma representaciones **esparsas** (vectores one-hot) en representaciones **densas** (embeddings) que capturan relaciones semánticas globales entre palabras.

---

### **Notación Preliminar**
1. **Vocabulario**: $ V $, el tamaño total del vocabulario.
2. **Dimensión del embedding**: $ d $, la dimensión del espacio donde proyectamos las palabras.
3. **Matrices de embedding**:
   - $ \mathbf{W} \in \mathbb{R}^{V \times d} $: Matriz de embeddings de entrada.
   - $ \mathbf{U} \in \mathbb{R}^{V \times d} $: Matriz de embeddings de salida.
4. **Vector one-hot**:
   - Cada palabra $ w $ se representa como $ \mathbf{x} \in \mathbb{R}^V $, donde $ x_i = 1 $ si $ i $ es el índice de $ w $, y $ x_j = 0 $ para $ j \neq i $.

---

## **CBOW en Lenguaje Matricial**

### **Forward Pass**

#### **Paso 1: Entrada - Palabras de Contexto**

Dada la frase:
$$
\text{"El gato persigue al ratón"}
$$

Queremos predecir $ w_t = \text{"persigue"} $, utilizando como contexto:
$$
\{\text{"El"}, \text{"gato"}, \text{"al"}, \text{"ratón"}\}.
$$

1. Asignamos índices al vocabulario:
$$
\{\text{"El"} \to 0, \text{"gato"} \to 1, \text{"persigue"} \to 2, \text{"al"} \to 3, \text{"ratón"} \to 4\}.
$$

2. Representamos las palabras de contexto como vectores one-hot:
$$
\mathbf{X} =
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 \\  % "El"
0 & 1 & 0 & 0 & 0 \\  % "gato"
0 & 0 & 0 & 1 & 0 \\  % "al"
0 & 0 & 0 & 0 & 1     % "ratón"
\end{bmatrix}
\in \mathbb{R}^{4 \times 5}.
$$

---

#### **Paso 2: Transformación - Embeddings densos**

La matriz de embeddings $ \mathbf{W} \in \mathbb{R}^{V \times d} $ transforma las representaciones esparsas en vectores densos. La operación es:
$$
\mathbf{E}_{\text{contexto}} = \mathbf{X} \mathbf{W}.
$$

Supongamos que $ d = 2 $ y:
$$
\mathbf{W} =
\begin{bmatrix}
0.1 & 0.2 \\  % "El"
0.3 & 0.4 \\  % "gato"
0.5 & 0.6 \\  % "persigue"
0.7 & 0.8 \\  % "al"
0.9 & 1.0     % "ratón"
\end{bmatrix}.
$$

Entonces:
$$
\mathbf{E}_{\text{contexto}} =
\begin{bmatrix}
0.1 & 0.2 \\  % "El"
0.3 & 0.4 \\  % "gato"
0.7 & 0.8 \\  % "al"
0.9 & 1.0     % "ratón"
\end{bmatrix}
\in \mathbb{R}^{4 \times 2}.
$$

Cada fila representa un embedding denso que captura las características semánticas latentes de cada palabra.

---

#### **Paso 3: Promedio de los embeddings**

Para resumir el contexto, calculamos el promedio de los vectores densos:
$$
\mathbf{v}_{\text{contexto}} = \frac{1}{2k} \sum_{i=1}^{2k} \mathbf{E}_{\text{contexto}}[i, :].
$$

En este caso:
$$
\mathbf{v}_{\text{contexto}} = \frac{1}{4} \begin{bmatrix} 0.1+0.3+0.7+0.9 & 0.2+0.4+0.8+1.0 \end{bmatrix} = \begin{bmatrix} 0.5 & 0.6 \end{bmatrix}.
$$

---

#### **Paso 4: Proyección al espacio del vocabulario**

Proyectamos $ \mathbf{v}_{\text{contexto}} $ al espacio del vocabulario utilizando la matriz de salida $ \mathbf{U} \in \mathbb{R}^{V \times d} $:
$$
\mathbf{z} = \mathbf{U} \mathbf{v}_{\text{contexto}}.
$$

Por ejemplo, si:
$$
\mathbf{U} =
\begin{bmatrix}
0.2 & 0.3 \\  % "El"
0.4 & 0.5 \\  % "gato"
0.6 & 0.7 \\  % "persigue"
0.8 & 0.9 \\  % "al"
1.0 & 1.1     % "ratón"
\end{bmatrix},
$$

entonces:
$$
\mathbf{z} =
\begin{bmatrix}
0.2 & 0.3 \\ 
0.4 & 0.5 \\ 
0.6 & 0.7 \\ 
0.8 & 0.9 \\ 
1.0 & 1.1
\end{bmatrix}
\cdot
\begin{bmatrix}
0.5 \\ 
0.6
\end{bmatrix}
=
\begin{bmatrix}
0.28 \\  % "El"
0.49 \\  % "gato"
0.70 \\  % "persigue"
0.91 \\  % "al"
1.12     % "ratón"
\end{bmatrix}.
$$

---

#### **Paso 5: Softmax**

Finalmente, aplicamos softmax para convertir $ \mathbf{z} $ en una distribución de probabilidad:
$$
P(w_t | \text{contexto}) = \frac{\exp(z_i)}{\sum_{j=1}^V \exp(z_j)}.
$$

La palabra con la mayor probabilidad será la predicción $ \hat{w}_t $, que debería ser "persigue".

---

### **3. Por qué CBOW Encuentra Relaciones Densas**

1. **De esparso a denso**:
   Transformar $ \mathbf{x} $ mediante $ \mathbf{W} $ reduce la dimensionalidad (de $ V $ a $ d $) y captura relaciones semánticas latentes.

2. **Relaciones semánticas globales**:
   Los embeddings densos reflejan la coocurrencia de palabras en contextos similares. Palabras como "gato" y "ratón", que aparecen juntas, tendrán vectores cercanos.

3. **Eficiencia computacional**:
   Al calcular un promedio, se reduce el impacto de palabras menos informativas como artículos y preposiciones.

---

### **4. Impacto en Semántica y Sintaxis**

1. **Semántica**:
   - CBOW captura relaciones globales. Por ejemplo, "gato" y "perro" tendrán embeddings cercanos si aparecen en contextos similares.

2. **Sintaxis**:
   - CBOW ignora el orden de las palabras. Por ejemplo, "El gato persigue al ratón" y "El ratón persigue al gato" tendrán la misma representación promedio, lo que puede ser una limitación en tareas sensibles al orden.


## **Skip-gram**

En Skip-gram, la dirección del problema se invierte: en lugar de predecir la palabra objetivo $ w_t $ a partir del contexto, ahora utilizamos $ w_t $ para predecir cada palabra del contexto. Esto significa que cada palabra central está asociada a múltiples predicciones, una para cada palabra en su ventana de contexto.

---

### **1. ¿Qué se optimiza en Skip-gram?**

El objetivo del modelo Skip-gram es **maximizar la probabilidad condicional de las palabras del contexto** $ \{w_{t-k}, \ldots, w_{t-1}, w_{t+1}, \ldots, w_{t+k}\} $, dado la palabra central $ w_t $. 

La pérdida del modelo está definida como la log-verosimilitud negativa sobre todas las palabras del contexto:
$$
\mathcal{L} = -\sum_{t=1}^T \sum_{j=-k, j \neq 0}^k \log P(w_{t+j} | w_t),
$$
donde:
- $ P(w_{t+j} | w_t) $ es la probabilidad predicha de que $ w_{t+j} $ sea una palabra del contexto, dado $ w_t $.
- $ T $ es el número total de palabras en el corpus.
- $ k $ es el tamaño de la ventana de contexto.

---

### **2. Probabilidad condicional en Skip-gram**

La probabilidad condicional $ P(w_{t+j} | w_t) $ se calcula utilizando la función softmax, de manera similar a CBOW:
$$
P(w_{t+j} | w_t) = \frac{\exp(\mathbf{u}_{w_{t+j}}^\top \mathbf{v}_{w_t})}{\sum_{i=1}^V \exp(\mathbf{u}_i^\top \mathbf{v}_{w_t})},
$$
donde:
- $ \mathbf{v}_{w_t} $ es el embedding de la palabra central $ w_t $, obtenido de la matriz $ \mathbf{W} $.
- $ \mathbf{u}_{w_{t+j}} $ es el embedding de salida de la palabra del contexto $ w_{t+j} $, tomado de la matriz $ \mathbf{U} $.
- $ V $ es el tamaño del vocabulario.

La optimización ajusta los parámetros de $ \mathbf{W} $ y $ \mathbf{U} $ para maximizar la probabilidad de las palabras reales del contexto dadas las palabras centrales.

---

### **3. ¿Cómo se originan las probabilidades en Skip-gram?**

Desglosemos este proceso con un ejemplo.

---

#### Ejemplo: **"El gato persigue al ratón"**

Queremos predecir el contexto $ \{\text{"El"}, \text{"gato"}, \text{"al"}, \text{"ratón"}\} $ dado $ w_t = \text{"persigue"} $.

---

### Paso 1: Representación One-Hot de la Palabra Central

Asignamos índices al vocabulario:
$$
\{\text{"El"} \to 0, \text{"gato"} \to 1, \text{"persigue"} \to 2, \text{"al"} \to 3, \text{"ratón"} \to 4\}.
$$

La palabra central $ w_t = \text{"persigue"} $ se representa como un vector one-hot:
$$
\mathbf{x}_{w_t} =
\begin{bmatrix}
0 \\  % "El"
0 \\  % "gato"
1 \\  % "persigue"
0 \\  % "al"
0     % "ratón"
\end{bmatrix}
\in \mathbb{R}^{5}.
$$

---

### Paso 2: Transformación a Embedding Denso

Multiplicamos $ \mathbf{x}_{w_t} $ por la matriz de embeddings de entrada $ \mathbf{W} \in \mathbb{R}^{V \times d} $ para obtener el embedding denso de la palabra central:
$$
\mathbf{v}_{w_t} = \mathbf{x}_{w_t}^\top \mathbf{W}.
$$

Supongamos que $ d = 2 $ y:
$$
\mathbf{W} =
\begin{bmatrix}
0.1 & 0.2 \\  % "El"
0.3 & 0.4 \\  % "gato"
0.5 & 0.6 \\  % "persigue"
0.7 & 0.8 \\  % "al"
0.9 & 1.0     % "ratón"
\end{bmatrix}.
$$

La multiplicación selecciona la fila correspondiente a $ w_t = \text{"persigue"} $:
$$
\mathbf{v}_{w_t} =
\begin{bmatrix}
0.5 & 0.6
\end{bmatrix}.
$$

---

### Paso 3: Proyección al Espacio del Vocabulario

Para cada palabra del contexto $ w_{t+j} $, calculamos las puntuaciones $ z_{w_{t+j}} $ mediante la proyección:
$$
z_i = \mathbf{u}_i^\top \mathbf{v}_{w_t},
$$
donde $ \mathbf{u}_i $ es el vector de salida de la palabra $ i $, tomado de la matriz $ \mathbf{U} \in \mathbb{R}^{V \times d} $.

Supongamos:
$$
\mathbf{U} =
\begin{bmatrix}
0.2 & 0.3 \\  % "El"
0.4 & 0.5 \\  % "gato"
0.6 & 0.7 \\  % "persigue"
0.8 & 0.9 \\  % "al"
1.0 & 1.1     % "ratón"
\end{bmatrix}.
$$

Calculamos las puntuaciones para todas las palabras:
$$
\mathbf{z} = \mathbf{U} \mathbf{v}_{w_t}.
$$

Realizando la multiplicación:
$$
\mathbf{z} =
\begin{bmatrix}
0.2 & 0.3 \\ 
0.4 & 0.5 \\ 
0.6 & 0.7 \\ 
0.8 & 0.9 \\ 
1.0 & 1.1
\end{bmatrix}
\cdot
\begin{bmatrix}
0.5 \\ 
0.6
\end{bmatrix}.
$$

1. Para "El":
   $$
   z_{\text{"El"}} = 0.2 \cdot 0.5 + 0.3 \cdot 0.6 = 0.1 + 0.18 = 0.28.
   $$
2. Para "gato":
   $$
   z_{\text{"gato"}} = 0.4 \cdot 0.5 + 0.5 \cdot 0.6 = 0.2 + 0.3 = 0.49.
   $$
3. Repetimos para todas las palabras:
   $$
   \mathbf{z} =
   \begin{bmatrix}
   0.28 \\  % "El"
   0.49 \\  % "gato"
   0.70 \\  % "persigue"
   0.91 \\  % "al"
   1.12     % "ratón"
   \end{bmatrix}.
   $$

---

### Paso 4: Conversión a Probabilidades (Softmax)

La función softmax normaliza estas puntuaciones:
$$
P(w_{t+j} | w_t) = \frac{\exp(z_{w_{t+j}})}{\sum_{i=1}^V \exp(z_i)}.
$$

1. Calculamos $ \exp(z_i) $:
   $$
   \exp(\mathbf{z}) =
   \begin{bmatrix}
   1.32 \\ 
   1.63 \\ 
   2.01 \\ 
   2.48 \\ 
   3.07
   \end{bmatrix}.
   $$

2. Calculamos la suma:
   $$
   \sum_{i=1}^V \exp(z_i) = 1.32 + 1.63 + 2.01 + 2.48 + 3.07 = 10.51.
   $$

3. Dividimos cada $ \exp(z_i) $ entre la suma para obtener probabilidades:
   $$
   P(w_{t+j} = \text{"El"} | w_t) = \frac{1.32}{10.51} \approx 0.126,
   $$
   $$
   P(w_{t+j} = \text{"gato"} | w_t) = \frac{1.63}{10.51} \approx 0.155,
   $$
   y así sucesivamente.

---

### **4. Relación entre Probabilidades y Optimización**

- Cada palabra del contexto tiene su propia probabilidad condicional $ P(w_{t+j} | w_t) $.
- El modelo ajusta los parámetros

## 4. Entrenamiento del modelo

Una vez que se ha preprocesado el texto, se puede entrenar el modelo Word2Vec. Para ello, se utiliza la clase `Word2Vec` de la librería Gensim. Se pueden configurar varios parámetros, como el tamaño del vector, la ventana de contexto, el número de iteraciones, etc.

In [ ]:
############# Entrenamiento de modelo Word2Vec ####################

modelo = Word2Vec(corpus_procesado, vector_size=100, window=5, min_count=1, sg=0)

### Aqui cada parametro significa lo siguiente:
# vector_size: Dimension de los vectores de palabras
# window: Numero de palabras que se toman en cuenta para predecir la siguiente palabra
# min_count: Frecuencia minima de palabras para ser considerada
# sg: 0 para CBOW y 1 para Skip-gram



print(modelo.wv.key_to_index)
print(modelo.wv["hielo"])


## 5. Exploración de las representaciones vectoriales

Una vez entrenado el modelo, se pueden explorar las representaciones vectoriales de las palabras. Por ejemplo, se pueden obtener las palabras más similares a una palabra dada, o realizar operaciones de álgebra de vectores para encontrar relaciones entre palabras.

In [ ]:
#### Explorar representaciones vectoriales de palabras ####

print(modelo.wv.most_similar("hielo"))

In [ ]:
print(modelo.wv.most_similar("padre"))

In [ ]:
### ALgebra de palabras ###

print(modelo.wv.most_similar(positive=["hielo", "padre"], negative=["perezoso"]))



## 6. Conclusiones

En este notebook, hemos visto cómo entrenar un modelo Word2Vec utilizando la librería Gensim y explorar las representaciones vectoriales de palabras obtenidas. Estas representaciones son útiles para realizar tareas de procesamiento de lenguaje natural, como la clasificación de texto, la traducción automática, la generación de texto, etc.